## Motivación 

Hay tres motivos principales por los que es necesario un estudio de los sistemas en tiempo real:

- Necesidad de análilis de datos cercano al tiempo real (baja latencia).
- Datasets masivos no delimitados (unbounded) -> datos que nunca finalizan.
- Carga de trabajo mejor distribuida (consistenciay predictibilidad).

## Casos de uso

- Sistemas de recomendación (redes sociales):
    - Millones de eventos por segundo
    - Respuestas en minutos
    
- Online banking (detección de fraudes):
    - Detección en segundos de actividad fraudulenta

- Facturación de llamadas (redes móviles):
    - Cientos de millones de usuarios
    - Cálculo en menos de un segundo
    - Cero tolerancia a fallos
    
    
## Terminología y clasificación

- Un ``stream de datos`` es una secuencia de elementos recibidos por un sistema en tiempo real, de forma continua y or4denada
    - Orden dado por un tiemstamp
    - Orden dado por una posición
- Existe la posibilidad de que controlar el orden en que se reciben los datos sea extremadamente difícil o imposible
- Consultas continuas -> Resultados incrementales
- Sistema de streaming: Sistemas de procesamiento de datos diseñado para el trabajo con datasets infinitos. En contraposición a sistemas batch.
- Cardinalidad de un dataset:
    - Delimitada (bounded): Dataset finito en tamaño
    - No delimitada (unbounded): Dataset (teóricamente infinito en tamaño
- Constitución de un dataset (manifestación física):
    - Tabla: Vista global del dataset en un momento determinado (ej: SQL)
    - Stream: Vista elemento a elemento de la evolución del dataset a lo largo del tiempo
- Tiempo de evento: Momento en el que ocurre el evento.
- Tiempo de proceso: Momento en el que el sistema observa y procesa el evento
- Motivos de distorsión (diferencia entre ambos):
    - Limitación de recursos distribuidos (congestión de red, CPU compartida, etc).
    - Causas software (lógica distribuida, contención, etc).
    - Características de los datos (desviaciones de rendimiento u orden, etc).
    
## Tiempo de evento vs tiempo de proceso

![streaming_1]
- La línea negra discontínua con pendiente 1 representa el ideal, donde el tiempo de procesamiento y el tiempo del evento son exactamente iguales.
- La línea roja representa la realidad.

En este ejemplo el sistema se retrasa unpoco al comienzo del tiempo de procesamiento, se acerca al ideal en el medio y luego vuelve a retrasarse un poco hacia el final.



## Estrategias de procesamiento de datos

- Datos delimitados:
    - Estrategia directa
    - Motor de procesamiento de datos basado en batchs
    - MapReduce
![streaming_2]
- Datos no delimitados:
    - Batch:
        - Ventanas fijas
            - Simula el tratamiento de datos delimitados dividiéndolos en ventanas de tamaño fijo
            - Problemas de integridad -> Retraso del procesado hasta tener todos los datos, o reprocesado si los datos llegan tarde.
        - Sesiones
            - Periodos de actividad entre tiempos de inactividad
            - Divididas entre batches -> incerementar el tamaño de batch -> mayor latencia
![streaming_3]
    - Streaming:
        - Estrategias específicas para datasets no delimitados
        - Desordenados respecto a tiempo de evento -> Necesidad de ordenación basada en el tiempo
        - Variación de la diferencia evento - proceso
        - Diferentes aproximaciones
- Time-agnostic streaming:
    - El tiempo no es una variable importante en los datos
    - Ejemplos:
        - Filtrado: comprobación de cada elemento y procesado de sólo los que nos interesan.
        - Inner joins: unión de dos fuentes de datos no delimitadas a partir de sus claves -> El primer elemento se almacena y la unión sólo se procesa cuando llega el segundo -> Problemas de integridad(¿qué ocurre si el segundo elemento nunca llega?¿hasta cuando se espera?
        
- Approximation streaming:
    - Basados en el tiempo de proceso
    - Ofrecen una salida aporximada a partir de los datos que van procesando.
    - Algoritmos complicados y escasos
    - Necesidad de buenas estimaciones de las cotas de error
    
- Windowing:
    - Ventanas fijas: segmentos uniformes en todo el dataset.
    - Ventanas deslizantes: definidas por una longitud y un período (salto entre ventanas).
    - Sesiones: ventanas dinámicas, que agrupan eventos relacionados en el tiempo. Sus longitudes no pueden ser definidad a priori.
![streaming_4]
    - El tiempo de proceso (processing-time windowing): transcurrido un tiempo concreto, el sistema procesa todos los datos que han llegado en dicha ventana.
        - Implementación sencilla
        - Análisis sencillo de la integridad: se sabe cuándo todos los datos de la ventana han llegado
        - Ejemplo Monitorización de actividad
        - Gran desventaja: Necesidad de gestionar los tiempos en los que ocurrió el evento.
    - En tiempo de evento (event-time windowing): las ventanas se han de organizar en función del tiempo en el que se produjo el evento.
        - Permiten ventanas dinámicas (sesiones)
        - Se requiere mucho más buffering de datos (almacenamiento persistente) -> Posibibilidad de almacenamiento agregado (sumas o medias de datos)

## Semátinca de entrega (delivery semantics)

- Se utiliza para definir el cumplimiento de la integridad de datos en el proceso de streaming
    - Datos perdidos
    - Datos duplicados
- Diversas estrategias
    - At-most-once
![streaming_5]
    - At-least-once
        - Cada mensaje se entrega al menos una vez
        - Ningún mensaje se pierde
        - Los mensajes pueden duplicarse
        - Casos de uso donde los duplicados no suponen un problema:
            - Visitas a un sitio web en las que se realiza un conteo de los usuarios únicos que han accedido (no se consideran los duplicados)
![streaming_6]
    - Exactly-once:
        - Cada mensaje se entrega exactamente una vez
        - Ningún mensaje se pierde
        - Ninún mensaje se duplica
        - Muy costoso de implementar (y no siempre necesario)
        - Escenarios más estrictos:
            - Operador móvil que calcula lo que ha de pagar el usuario (no puede perder dinero ni duplicar ningún coste)
![streaming_7]

## Transformaciones semánticas

- At-least-once -> Exactly-once
    - Deduplicación de mensajes en el sistema final
        - Identificador explícito / calculable a partir de los datos.
        - Aumenta la latencia
        - No aplicable en el procesamiento en tiempo real
    - Idempotencia
        - Propiedad de una operación que puede aplicarse muchas veces sin modificar el estado final: f(f(x)) = f(x)
        - Ejemplos: Valor absoluto, añadir un elemento a un conjunto
        - Ejemplo en streaming: Añadir un nuevo dominio visitado por un usuario al conjunto de dominios ya visitados.
        - No siempre aplicable-
        
## Arquitectura Lambda y Kappa

- Arquitectura Lambda
![streaming_8]
    - Capa Batch: Genera las vistas que se producen en el procesamiento batch, y las indexa (serving layer) para reducir la latencia en la consulta
    - Capa Streaming (Speed layer): sólo considera datos nuevos.
    - La respuesta final es una combinación de ambas capas.
    - Combina procesamiento batch y streaming
    - Se duplica la funcionalidad de tiempo real en la capa offline
    - Más universal
    - Pros
        - Re-cálculo de datos y correción de errores sencilla
        - Capacidad de procesar grandes volúmenes de datos en retrospectiva
        - Permite resolver problemas con lógica clásica basada en batch
    - Cons
        - Necesidad de duplicar la funcionalidad (dos sistemas)
        - No todos los problemas requieren una arquitectura lambda
- Arquitectura Kappa
![streaming_9]
    - La capa de streaming se ocupa también de datos históricos
    - Optimiza el uso de recursos (más minimalistas)
    - Muchos problemas son resolubles utilizando únicamente la arquitectura Kappa
    - Pros
        - Permite re-cálculo de datos y correción de errores
        - Única implementación
        - Procesamiento con retrospectiva
    - Cons
        - La retrospectiva que permite es limitada
        - No permite resolver problemas que no se puedan expresar como streaming
        - Los datos históricos no pueden variar
        - El almacenamiento de datos es más limitado

## Bibliografía
- Estos apuntes están basados en:
    - Streaming Systems. Capítulo 1 - Streaming 101
    - Stream processing with Apache Spark. Capítulo 3 - Streaming Architectures

[streaming_1]:images/streaming_1.png
[streaming_2]:images/streaming_2.png
[streaming_3]:images/streaming_3.png
[streaming_4]:images/streaming_4.png
[streaming_5]:images/streaming_5.png
[streaming_6]:images/streaming_6.png
[streaming_7]:images/streaming_7.png
[streaming_8]:images/streaming_8.png
[streaming_9]:images/streaming_9.png